# Lecture 2: Word Vectors, Word Senses, and Neural Network Classifiers

## The skip-gram model with negative sampling

Objective function:

$$
\begin{align*}
J_t(\theta) &= \log \sigma (u_oTv_c) + \sum_{i=1}^{k}\mathbb{E}_{j\sim P(w)} \left[\log \sigma (-u_j^Tv_c)\right]\\
J(\theta) &= \frac{1}{T}\sum_{t=1}^{T}J_t(\theta)
\end{align*} 
$$

---

Co-ocurrence matrix 问题：

1. 直接使用 raw counting matrix 维度过高；但在这个场景下，降维方法例如 SVD 效果不好
2. 一些单词出现频率较高但没啥用，例如 "he", "I", "her" 等

---

改进：

1. 在进行 negative sampling 时，希望出现频率更少的单词更有可能被选中为 negative sample，可以选择适当的抽样分布来实现。
2. 在计算 co-ocurrence matrix 时，可以考虑多种处理 counting 的方式，例如取对数、限制上界、忽略部分单词、添加距离信息、换用 Pearson correlation 等

## GloVe

- 使用 **weighted least squares model**

### Co-occurrence Matrix

- $X$: co-occurrence matrix
- $X_i = \sum_{k}X_{ik}$: 在单词 $i$ 附近出现的总单词数
- $P_{ij} = P(w_j | w_i) = \frac{X_{ij}}{X_i}$: 单词 $j$ 出现在 $i$ 的 context 中的概率

### Least Squares Objective

在 skip-gram model 中，单词 $j$ 出现在 $i$ 的 context 中的概率

$$
Q_{ij} = \frac{\exp (u_j^Tv_i)}{\sum_{w=1}^{W}\exp (u_w^Tv_i)}
$$

Global cross-entropy loss:

$$
\begin{align*}
    J&=-\sum_{i \in corpus} \sum_{j \in context(i)} \log Q_{ij} \\
    &= - \sum_{i=1}^{W}\sum_{j=1}^{W}X_{ij}\log Q_{ij}
\end{align*} 
$$

使用这个 loss function 的缺点在计算量太大，$Q$ 需要计算所有单词的余弦相似度并取指数。解决方法：使用 least suqare objective，丢弃掉 $P$ 和 $Q$ 的 normalization factors:

$$
\hat{J} = \sum_{i=1}^{W}\sum_{j=1}^{W} X_i(\hat{P}_{ij} - \hat{Q}_{ij})^2
$$

这里

- $\hat{P}_{ij} = X_{ij}$
- $\hat{Q}_{ij} = \exp (u_j^Tv_i)$

但这样的话又有一个问题：$X_{ij}$ 通常会很大，不好优化。可以通过取对数来缓解：

$$
\begin{align*}
    \hat{J} &= \sum_{i=1}^{W}\sum_{j=1}^{W} X_i(\log (\hat{P}_{ij})_ - \log (\hat{Q}_{ij}))^2 \\
    &= \sum_{i=1}^{W} \sum_{j=1}^{W}X_i(u_j^Tv_i - \log X_{ij})^2
\end{align*} 
$$

又权重 $X_i$ 不一定是最优的权重，可以替换为函数 $f(X_ij)$:

$$
\hat{J} = \sum_{i=1}^{W}\sum_{j=1}^{W}f(X_{ij})(u_j^Tv_i - \log X_{ij})^2
$$

## Evaluation of Word Vectors

产生的 word vectors 也需要被测试质量高低。

### Intrinsic Evaluation

使用一些中间的子任务（例如 analogy completion 等）来评测，而不是直接使用最终的任务（即 extrinsic evaluation）。

例如，如果想搭建一个问答系统，那么可以用 analogy completion 子任务来代替最终任务进行训练：

![](https://cdn.jsdelivr.net/gh/KinnariyaMamaTanha/Images@main/202408221313797.png)

对于 subtask `a:b :: c:?`，即从条件 `a -> b` 得到 `c` 对应的 vector index `d`：

$$
d = \operatorname{argmax}_i \frac{(x_b - x_a + x_c)^T x_i}{\left\| x_b - x_a + x_c \right\| }
$$